![image](https://raw.githubusercontent.com/Trouble404/kaggle-Job-Salary-Prediction/master/readme_pic/describtion.PNG)

# Load Dataset 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [14]:
First_Valid = pd.read_csv('dataset\Valid_rev1.csv')
First_Valid.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SourceName
0,13656201,"Lead Technical Architect, C Banking",Lead Technical Architect required for a Tier *...,London,London,NaN,permanent,Scope AT Limited,IT Jobs,jobserve.com
1,14663195,RECRUITMENT CONSULTANT INDUSTRIAL / COMMERCIA...,THIS IS A LIVE VACANCY NOT A GENERIC ADVERTISE...,"LEEDS, West Yorkshire",Leeds,NaN,permanent,Code Blue Recruitment,HR & Recruitment Jobs,cv-library.co.uk
2,16530664,Mechanical / Chemical / Process Engineer Cool...,Mechanical / Chemical / Process Engineer Cool...,"Hampshire, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
3,19047458,Trainee Mortgage Advisor West Midlands,"Are you a successful, results driven person? A...",West Midlands,West Midlands,NaN,permanent,Brite Recruitment,Accounting & Finance Jobs,cv-library.co.uk
4,20881907,Mortgage Services Consultant East Midlands,"Are you a successful, results driven person? D...",East Midlands,East Midlands,NaN,permanent,Brite Recruitment,Accounting & Finance Jobs,cv-library.co.uk


** What's the use of valid data? Difference between trian data?**

In [7]:
First_Train = pd.read_csv('dataset\Train_rev1.csv')
First_Train.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk


**Train dataset contains 244768 rows × 12 columns**

* Id - A unique identitier for each job ad.
* Title - A freetext field supplied to us by the job advertiser as the Title of the job ad.
* LocationRaw - The freetext location as provided by the job advertiser.
* LocationNormalized - Adzuna's normalised location from within our own location tree, interpreted by us based on the raw location. This normaliser is not **perfect**.
* ContractType - full_time or part_time, interpreted by Adzuna from description or a specific additional field we received from the advertiser.
* ContractTime - permanent or contract, interpreted by Adzuna from description or a specific additional field we received from the advertiser.
* Company - the name of the employer as supplied to us by the job advertiser.
* Category - which of 30 standard job categories this ad fits into, inferred in a very messy way based on the source the ad came from. There is a lot of **noise** and **error** in this field.
* SalaryRaw - the freetext salary field we received in the job advert from the advertiser.
* SalaryNormalised - the annualised salary interpreted by Adzuna from the raw salary.  Note that this is always a single value based on the midpoint of any range found in the raw salary.  This is the value we are trying to predict.
* SourceName - the name of the website or advertiser from whom we received the job advert. 

In [9]:
First_Test = pd.read_csv('dataset\Test_rev1.csv')
First_Test.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SourceName
0,11888454,Business Development Manager,The Company: Our client is a national training...,"Tyne Wear, North East",Newcastle Upon Tyne,NaN,permanent,Asset Appointments,Teaching Jobs,cv-library.co.uk
1,11988350,Internal Account Manager,The Company: Founded in **** our client is a U...,"Tyne and Wear, North East",Newcastle Upon Tyne,NaN,permanent,Asset Appointments,Consultancy Jobs,cv-library.co.uk
2,12612558,Engineering Systems Analysts,Engineering Systems Analysts Surrey ****K Loca...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
3,12613014,CIS Systems Engineering Consultant,CIS Systems Engineering Consultant Bristol So...,"Bristol, South West, South West",Bristol,NaN,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
4,22454872,CNC Miller / Programmer Fanac,"CNC Miller / Programmer Fanac Fleet, Hampshire...","Fleet, Hampshire",Fleet,NaN,permanent,Gregory Martin International,Manufacturing Jobs,cv-library.co.uk


**Test dataset contains 122463 rows × 10 columns**

It will be used to predict the **SalaryRaw** and **SalaryNormalised**

In [84]:
Location_Tree = pd.read_csv('dataset\Location_Tree.csv')
Location_Tree.head()

,FullLocation
0,UK~London~East London~Mile End
1,UK~London~East London~Shadwell
2,UK~London~East London~Spitalfields
3,UK~London~East London~Stepney
4,UK~London~East London~Wapping


This is a supplemental data set that describes the hierarchical relationship between the different Normalised Locations shown in the job data.  It it is likely that there are meaningful relationships between the salaries of jobs in a similar geographical area, for example average salaries in London and the South East are higher than in the rest of the UK.

# Feature engineering

In [10]:
# Train dataset
First_Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 12 columns):
Id                    244768 non-null int64
Title                 244767 non-null object
FullDescription       244768 non-null object
LocationRaw           244768 non-null object
LocationNormalized    244768 non-null object
ContractType          65442 non-null object
ContractTime          180863 non-null object
Company               212338 non-null object
Category              244768 non-null object
SalaryRaw             244768 non-null object
SalaryNormalized      244768 non-null int64
SourceName            244767 non-null object
dtypes: int64(2), object(10)
memory usage: 22.4+ MB


In [11]:
# Test dataset
First_Test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122463 entries, 0 to 122462
Data columns (total 10 columns):
Id                    122463 non-null int64
Title                 122463 non-null object
FullDescription       122463 non-null object
LocationRaw           122463 non-null object
LocationNormalized    122463 non-null object
ContractType          33013 non-null object
ContractTime          90702 non-null object
Company               106202 non-null object
Category              122463 non-null object
SourceName            122463 non-null object
dtypes: int64(1), object(9)
memory usage: 9.3+ MB


It could find there are missing value in **ContractType**, **ContractTime** and **Company**

# clustering by openrefine-2.8

**combine train and test datasets**

**cluster data in each cloumn**

**fill missing value (not all)**

![iamge](https://raw.githubusercontent.com/Trouble404/kaggle-Job-Salary-Prediction/master/readme_pic/cleaning_step1.PNG)

In [2]:
Second_test = pd.read_csv('dataset/salay_first_dataset.csv')
Second_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367231 entries, 0 to 367230
Data columns (total 13 columns):
File                  367231 non-null object
Id                    367231 non-null int64
Title                 367231 non-null object
FullDescription       367231 non-null object
LocationRaw           367231 non-null object
LocationNormalized    367231 non-null object
ContractType          367231 non-null int64
ContractTime          367231 non-null int64
Company               318540 non-null object
Category              367231 non-null int64
SourceName            367231 non-null object
SalaryRaw             244768 non-null object
SalaryNormalized      244768 non-null float64
dtypes: float64(1), int64(4), object(8)
memory usage: 36.4+ MB


**Title**

In [7]:
title_counts = Second_test['Title'].value_counts()
print (title_counts)

Business Development Manager                                           1403
Project Manager                                                        1134
Management Accountant                                                   998
Sales Executive                                                         921
Cleaner                                                                 852
Account Manager                                                         830
Mechanical Design Engineer                                              728
Finance Manager                                                         694
Administrator                                                           690
Recruitment Consultant                                                  670
Credit Controller                                                       663
Accounts Assistant                                                      655
Assistant Manager                                                       644
Financial Co

In [28]:
title_name = []
for n in range(0,len(title_counts)):
    if title_counts[n] > 9:
        title_name.append(title_counts.index[n])

print(len(title_name))

2837


In [29]:
title = Second_test['Title']
#title_name = title_counts.index
title_vec = np.arange(0,len(title_name),1) + 1

In [48]:
title_1 = []
for title_each in tqdm(title):
    test1 = -1
    for n in range(0,len(title_name)):
        if title_each == title_name[n]:
            test1 = n
    if test1 == -1:
        title_1.append(title_each)
    else:
        title_1.append(title_vec[test1])    

100%|████████████████████████████████████████████████████████████████████████| 367231/367231 [02:04<00:00, 2942.56it/s]


In [37]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False

In [49]:
for n in range(0,len(title_1)):
    if is_number(title_1[n]):
        title_1[n] = title_1[n]
    else:
        title_1[n] = 0

In [50]:
check = pd.DataFrame({'title': title_1})
check.to_csv('title_1.csv')

** Location**

In [89]:
location_tree = Location_Tree['FullLocation']

In [8]:
location_1 = Second_test['LocationRaw']
location_2 = Second_test['LocationNormalized']

In [57]:
# obtain max lenght string location
location_all = []
for n in tqdm(range(0,len(location_1))):
    location_split1 = location_1[n].split(',')
    
    for j in range(0,len(location_split)):
        location_split2 = location_split1[j].split('/')
        location_split3 = location_split2[j].split('/')
        location_split4 = location_split3[j].split('-')
        
    location_split4.insert(0,location_2[n])
    max_location = max(location_split4, key=len)
    location_all.append(max_location)

100%|███████████████████████████████████████████████████████████████████████| 367231/367231 [00:10<00:00, 35579.24it/s]


In [90]:
location_3 = []
for location_each in tqdm(location_all):
    test1 = -1
    for n in range(0,len(location_tree)):
        if location_each in location_tree[n]:
            test1 = n
    if test1 == -1:
        location_3.append(location_each)
    else:
        location_3.append(location_tree[test1])  

100%|███████████████████████████████████████████████████████████████████████| 367231/367231 [39:33:46<00:00,  2.58it/s]


In [91]:
check1 = pd.DataFrame({'location': location_3})
check1.to_csv('location_1.csv')

In [93]:
check1.head()

,location
0,UK~North East England~Tyne & Wear~Newcastle Up...
1,UK~North East England~Tyne & Wear~Newcastle Up...
2,UK~South East England~Surrey~Richmond Upon Thames
3,UK~South West England~Somerset~Bridgwater~Stoc...
4,UK~North East England~Northumberland~Chathill~...


In [95]:
count = 0
for locationTree in location_3:
    if '~' in locationTree:
        count = count + 1
count
        

248257

In [12]:
location_3 = pd.read_csv('location_1.csv')['location']

**1/3 location can't match full location in LocationTree**

In [13]:
# obtain max lenght string location
location_10 = []
location_11 = []
location_12 = []
location_13 = []
location_14 = []
location_15 = []
for each in tqdm(location_3):
    location_each = each.split('~')
    

    if len(location_each) == 6:
        location_10.append(location_each[0])
        location_11.append(location_each[1])
        location_12.append(location_each[2])
        location_13.append(location_each[3])
        location_14.append(location_each[4])
        location_15.append(location_each[5])

    if len(location_each) == 5:
        if location_each[0] == 'UK':
            location_10.append(location_each[0])
        else:
            location_10.append('')
            
        location_11.append('')
        location_12.append(location_each[1])
        location_13.append(location_each[2])
        location_14.append(location_each[3])
        location_15.append(location_each[4])

    if len(location_each) == 4:
        if location_each[0] == 'UK':
            location_10.append(location_each[0])
        else:
            location_10.append('')
            
        location_11.append('')
        location_12.append('')
        location_13.append(location_each[1])
        location_14.append(location_each[2])
        location_15.append(location_each[3])

    if len(location_each) == 3:
        if location_each[0] == 'UK':
            location_10.append(location_each[0])
        else:
            location_10.append('')
            
        location_11.append('')
        location_12.append('')
        location_13.append('')
        location_14.append(location_each[1])
        location_15.append(location_each[2])
            
    if len(location_each) == 2:
        if location_each[0] == 'UK':
            location_10.append(location_each[0])
        else:
            location_10.append('')
            
        location_11.append('')
        location_12.append('')
        location_13.append('')
        location_14.append('')
        location_15.append(location_each[1])          

    if len(location_each) == 1:
        if location_each[0] == 'UK':
            location_10.append(location_each[0])
            location_15.append('')
        else:
            location_10.append('')
            location_15.append(location_each[0])
            
        location_11.append('')
        location_12.append('')
        location_13.append('')
        location_14.append('')
                                   

100%|██████████████████████████████████████████████████████████████████████| 367231/367231 [00:00<00:00, 381261.38it/s]


In [ ]:
# when transorm on-hot vector, there are too many types of company, can't transfom, therefore, reduce the types of companys

In [16]:
fixed1 = pd.DataFrame({'location': location_15})

In [17]:
location_6_fixed = fixed1['location']

In [18]:
location_6_fixed_counts = fixed1['location'].value_counts()
location_6_fixed_counts

City of London                                     34832
South East London                                  16177
Newcastle-under-Lyme                               12807
Greater Manchester                                  5070
City London South East                              4266
Portway                                             4156
West End                                            3815
City of London                                      3451
Greater London                                      3409
Stockland Bristol                                   3306
Richmond Upon Thames                                2909
Methley                                             2895
Manchester Lancashire North West                    2805
Cambridgeshire                                      2803
Kingston                                            2575
Derriaghy                                           2508
Oxfordshire                                         2354
Nottinghamshire                

In [21]:
location_name = []
for n in range(0,len(location_6_fixed_counts)):
    if location_6_fixed_counts[n] > 9:
        location_name.append(location_6_fixed_counts.index[n])

print(len(location_name))

2087


In [22]:
#company_name = company_counts.index
location_vec = np.arange(0,len(location_name),1) + 1

In [25]:
location_1 = []
for location_each in tqdm(location_6_fixed):
    test1 = -1
    for n in range(0,len(location_name)):
        if location_each == location_name[n]:
            test1 = n
            break
    if test1 == -1:
        location_1.append(0)
    else:
        location_1.append(location_vec[test1])    

100%|███████████████████████████████████████████████████████████████████████| 367231/367231 [00:13<00:00, 27473.72it/s]


In [27]:
fiexed_location = pd.DataFrame({'location_6': location_1})
fiexed_location = fiexed_location.fillna(0)
fiexed_location.to_csv('location_1_fixed.csv')

In [165]:
check2 = pd.DataFrame({'location_1': location_10,
                      'location_2': location_11,
                      'location_3': location_12,
                      'location_4': location_13,
                      'location_5': location_14,
                      'location_6': location_15})
check2.head()

,location_1,location_2,location_3,location_4,location_5,location_6
0,UK,North East England,Tyne & Wear,Newcastle Upon Tyne,Woolsington,Newcastle Airport
1,UK,North East England,Tyne & Wear,Newcastle Upon Tyne,Woolsington,Newcastle Airport
2,UK,,,South East England,Surrey,Richmond Upon Thames
3,UK,,South West England,Somerset,Bridgwater,Stockland Bristol
4,UK,,North East England,Northumberland,Chathill,West Fleetham


In [173]:
check2.to_csv('location.csv')

In [172]:
cla_1 = {label:idx for idx,label in enumerate(np.unique(check2['location_1']))}
cla_2 = {label:idx for idx,label in enumerate(np.unique(check2['location_2']))}
cla_3 = {label:idx for idx,label in enumerate(np.unique(check2['location_3']))}
cla_4 = {label:idx for idx,label in enumerate(np.unique(check2['location_4']))}
cla_5 = {label:idx for idx,label in enumerate(np.unique(check2['location_5']))}
cla_6 = {label:idx for idx,label in enumerate(np.unique(check2['location_6']))}

In [174]:
check2['location_1'] = check2['location_1'].map(cla_1)
check2['location_2'] = check2['location_2'].map(cla_2)
check2['location_3'] = check2['location_3'].map(cla_3)
check2['location_4'] = check2['location_4'].map(cla_4)
check2['location_5'] = check2['location_5'].map(cla_5)
check2['location_6'] = check2['location_6'].map(cla_6)
check2.head()

,location_1,location_2,location_3,location_4,location_5,location_6
0,1,3,63,135,1219,5130
1,1,3,63,135,1219,5130
2,1,0,0,185,1066,5976
3,1,0,54,183,177,6793
4,1,0,38,152,250,8128


In [175]:
check2.to_csv('location_vec.csv')

**Company**

In [3]:
company = Second_test['Company']

In [4]:
# when transorm on-hot vector, there are too many types of company, can't transfom, therefore, reduce the types of companys

In [5]:
company_counts = Second_test['Company'].value_counts()
company_counts

UKStaffsearch                           7506
CVbrowser                               4456
London4Jobs                             3539
Hays                                    2883
JAM Recruitment Ltd                     2310
Matchtech Group plc.                    1681
Office Angels                           1440
Jobsite Jobs                            1385
ARRAY                                   1319
JOBG8                                   1296
Perfect Placement                       1273
Penguin Recruitment                     1140
Randstad                                1114
Adecco                                  1037
Michael Page Finance                     977
Interaction Recruitment                  938
Adecco Group                             870
COREcruitment International              865
BMS Sales Specialists LLP                845
Support Services Group                   804
Page Personnel Finance                   771
Michael Page Sales                       742
Capita Res

In [7]:
company_name = []
for n in range(0,len(company_counts)):
    if company_counts[n] > 19:
        company_name.append(company_counts.index[n])

print(len(company_name))

3037


In [8]:
#company_name = company_counts.index
company_vec = np.arange(0,len(company_name),1) + 1

In [28]:
company_1 = []
for company_each in tqdm(company):
    test1 = -1
    for n in range(0,len(company_name)):
        if company_each == company_name[n]:
            test1 = n
            break
    if test1 == -1:
        company_1.append(0)
    else:
        company_1.append(company_vec[test1])    

100%|████████████████████████████████████████████████████████████████████████| 367231/367231 [01:04<00:00, 5704.87it/s]


In [29]:
check3 = pd.DataFrame({'company': company_1})
check3 = check3.fillna(0)
check3.to_csv('company_1_fixed.csv')

In [11]:
check3.head()

,company
0,1256
1,1256
2,912
3,912
4,912


**SourceName**

In [219]:
sourcename = Second_test['SourceName']

In [220]:
source_cla_1 = {label:idx for idx,label in enumerate(np.unique(sourcename))}

In [224]:
sourcename_vec = sourcename.map(source_cla_1)

In [226]:
check4 = pd.DataFrame({'sourcename': sourcename_vec})
check4.to_csv('sourcename_1.csv')

In [228]:
check4.head()

,sourcename
0,42
1,42
2,42
3,42
4,42


# FullDescription 

In [3]:
describution = Second_test['FullDescription']

In [4]:
from sklearn.feature_extraction.text import HashingVectorizer

In [15]:
vectorizer = HashingVectorizer(n_features=500,lowercase=True,stop_words='english')

In [16]:
vector = vectorizer.transform(tqdm(describution))

100%|████████████████████████████████████████████████████████████████████████| 367231/367231 [01:21<00:00, 4532.93it/s]


In [17]:
vector

<367231x500 sparse matrix of type '<class 'numpy.float64'>'
	with 34716166 stored elements in Compressed Sparse Row format>

In [19]:
describution_full = []
for des_each in tqdm(vector):
    describution_full.append(des_each.toarray())  

367231it [00:22, 16467.80it/s]


In [20]:
check5 = pd.DataFrame({'fulldescribution': describution_full})
check5.head()

,fulldescribution
0,"[[-0.07738232325341368, 0.07738232325341368, 0..."
1,"[[0.03646624787447364, -0.03646624787447364, 0..."
2,"[[-0.09284766908852593, 0.0, 0.0, 0.0, 0.0, 0...."
3,"[[0.0, 0.0, -0.03763549647474907, -0.037635496..."
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


# using keras's onehot vector

In [36]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

In [28]:
# integer encode the documents 独热编码
vocab_size = 500
encoded_docs = [one_hot(d, vocab_size) for d in tqdm(describution)]

100%|████████████████████████████████████████████████████████████████████████| 367231/367231 [01:08<00:00, 5359.27it/s]


In [35]:
maxlength = len(max(encoded_docs, key=len))
maxlength

2081

In [39]:
padded_docs = pad_sequences(encoded_docs, maxlen = maxlength, padding='post')

In [43]:
padded_docs.shape

(367231, 2081)

In [44]:
padded_docs

array([[ 97, 128,  35, ...,   0,   0,   0],
       [ 97, 128, 394, ...,   0,   0,   0],
       [219, 163, 124, ...,   0,   0,   0],
       ...,
       [372, 165, 388, ...,   0,   0,   0],
       [372, 165, 388, ...,   0,   0,   0],
       [143, 483,  52, ...,   0,   0,   0]])

# Combine

In [47]:
%%time
file = Second_test['File']
Id = Second_test['Id']

df = pd.read_csv('title_1.csv')
title = df['title']

full_describution = Second_test['FullDescription']

df = pd.read_csv('location_vec.csv')
location_1 = df['location_1']
location_2 = df['location_2']
location_3 = df['location_3']
location_4 = df['location_4']
location_5 = df['location_5']
location_6 = df['location_6']

contracttype = Second_test['ContractType']
contracttime = Second_test['ContractTime']


df = pd.read_csv('company_1.csv')
company = df['company']

Category = Second_test['Category']

df = pd.read_csv('sourcename_1.csv')
sourcename = df['sourcename']

salaryraw = Second_test['SalaryRaw']
salaryNormalized = Second_test['SalaryNormalized']

Wall time: 565 ms


In [49]:
final = pd.DataFrame({'File': file,
                      'Id': Id,
                      'title': title,
                      'FullDescription': full_describution,
                      'location_1': location_1,
                      'location_2': location_2,
                      'location_3': location_3,
                      'location_4': location_4,
                      'location_5': location_5,
                      'location_6': location_6,
                      'ContractType': contracttype,
                      'ContractTime': contracttime,
                      'company': company,
                      'Category': Category,
                      'sourcename': sourcename,
                      'SalaryRaw': salaryraw,
                      'SalaryNormalized': salaryNormalized                                         
                     })
final.head()

,Category,ContractTime,ContractType,File,FullDescription,Id,SalaryNormalized,SalaryRaw,company,location_1,location_2,location_3,location_4,location_5,location_6,sourcename,title
0,26,1,0,Test_rev1.csv,The Company: Our client is a national training...,11888454,NaN,NaN,1265.0,1,3,63,135,1219,5130,42,1
1,3,1,0,Test_rev1.csv,The Company: Founded in **** our client is a U...,11988350,NaN,NaN,1265.0,1,3,63,135,1219,5130,42,495
2,8,1,0,Test_rev1.csv,Engineering Systems Analysts Surrey ****K Loca...,12612558,NaN,NaN,915.0,1,0,0,185,1066,5976,42,0
3,8,1,0,Test_rev1.csv,CIS Systems Engineering Consultant Bristol So...,12613014,NaN,NaN,915.0,1,0,54,183,177,6793,42,0
4,17,1,0,Test_rev1.csv,"CNC Miller / Programmer Fanac Fleet, Hampshire...",22454872,NaN,NaN,915.0,1,0,38,152,250,8128,42,0


Based on File name: split to trian and test data
full desciption will code in next part

In [51]:
final.to_csv('salary_vec.csv')

In [54]:
train = pd.read_csv('salary_train.csv')
train.head()

,Category,ContractTime,ContractType,FullDescription,Id,SalaryNormalized,SalaryRaw,company,location_1,location_2,location_3,location_4,location_5,location_6,sourcename,title
0,8,1,0,Engineering Systems Analyst Dorking Surrey Sal...,12612628,25000,20000 - 30000/annum 20-30K,915,1,0,53,201,353,5888,42,0
1,8,1,0,Stress Engineer Glasgow Salary **** to **** We...,12612830,30000,25000 - 35000/annum 25-35K,915,1,0,0,0,962,3062,42,0
2,8,1,0,Mathematical Modeller / Simulation Analyst / O...,12612844,30000,20000 - 40000/annum 20-40K,915,1,0,0,0,1004,3324,42,0
3,8,1,0,Engineering Systems Analyst / Mathematical Mod...,12613049,27500,25000 - 30000/annum 25K-30K negotiable,915,1,0,0,185,1066,5976,42,0
4,8,1,0,"Pioneer, Miser Engineering Systems Analyst Do...",12613647,25000,20000 - 30000/annum 20-30K,915,1,0,0,185,1066,5976,42,0


In [55]:
test = pd.read_csv('salary_test.csv')
test.head()

,Category,ContractTime,ContractType,FullDescription,Id,SalaryNormalized,SalaryRaw,company,location_1,location_2,location_3,location_4,location_5,location_6,sourcename,title
0,26,1,0,The Company: Our client is a national training...,11888454,NaN,NaN,1265,1,3,63,135,1219,5130,42,1
1,3,1,0,The Company: Founded in **** our client is a U...,11988350,NaN,NaN,1265,1,3,63,135,1219,5130,42,495
2,8,1,0,Engineering Systems Analysts Surrey ****K Loca...,12612558,NaN,NaN,915,1,0,0,185,1066,5976,42,0
3,8,1,0,CIS Systems Engineering Consultant Bristol So...,12613014,NaN,NaN,915,1,0,54,183,177,6793,42,0
4,17,1,0,"CNC Miller / Programmer Fanac Fleet, Hampshire...",22454872,NaN,NaN,915,1,0,38,152,250,8128,42,0
